In [2]:
from docx import Document
import time
from OCRDataGenerator import OCRDataGenerator
from datasets import load_dataset
import re
import unicodedata
from shannlp import word_tokenize


In [3]:
import time
from functools import wraps

def measure_time(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        # Record start time
        start_time = time.time()
        
        # Execute the function
        result = func(*args, **kwargs)
        
        # Record end time
        end_time = time.time()
        
        # Calculate and print execution time
        execution_time = end_time - start_time
        print(f"Function '{func.__name__}' took {execution_time:.4f} seconds to execute")
        
        return result
    return wrapper

## Extract Docx text

In [ ]:
def extract_docx_text(docx_path):
    doc = Document(docx_path)
    all_text = []
    buffer_text = ""

    for para in doc.paragraphs:
        words = para.text.split(" ")
        
        for word in words:
            if len(buffer_text) + len(word) + 1 > 30:
                all_text.append(buffer_text.strip())
                buffer_text = word
            else:
                buffer_text += " " + word
    
    if buffer_text:
        all_text.append(buffer_text.strip())
        
    return all_text


In [ ]:
def generate_images_from_docx(docx_path, fonts, output_dir):
    generator = OCRDataGenerator(font_paths=fonts)

    texts = extract_docx_text(docx_path)
    
    for text in texts:
        text = text.strip()
        
        if len(text) < 1:
            continue

        image, metadata = generator.generate_image(
            text=text,
            min_font_size=24,
            max_font_size=48,
            horizontal_padding=40,
            vertical_padding=20,
            min_height=64,
            add_noise=False,
            random_transform=False
        )
        print(f"Text: {text}")
        print(f"Image size: {metadata['image_size']}\n")

        ts = time.time()

        # save TIF
        image.save(f"{output_dir}/{ts}.tif")
            
        # save TXT
        with open(f"{output_dir}/{ts}.gt.txt", "w", encoding='utf-8') as text_file:
            text_file.write(text)

        print(f"Saved image for word: {text}")

In [ ]:
from pathlib import Path

datasets = Path("./kawtai-dataset")
output_dir = "./output"
fonts = [
    "./Shan.ttf",
    "./PangLong.ttf"
]

for file_path in datasets.rglob("*.docx"):
    if file_path.is_file():
        docx_path = file_path
        generate_images_from_docx(docx_path, fonts, output_dir)

## Extract from huggingface

In [30]:
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
        "]+", re.UNICODE)
    return re.sub(emoj, '', data)

def clean_latin_text(text):
    text = re.sub(r"[^\u1000-\u109f\s]", '', text)
    text = re.sub(r"\s+", " ", text)
    return text

def clean_shan_text(text, keep_numbers=False):
    text = text.replace("၊", "၊ ").replace("။", "။ ").replace(" ၊", "၊ ").replace(" ။", "။ ").strip()
    text = re.sub(r"ႉ{2,}", "ႉ", text)
    text = text.replace("ႆၢ", "ၢႆ")
    text = text.replace("ေတ", "တေ")

    text = clean_latin_text(text)
    
    # Core Shan-specific characters (characters that are uniquely Shan)
    shan_specific_chars = (
        # Shan consonants that are unique to Shan
        "\u1075\u1076\u1077\u1078\u1079\u107A\u107B\u107C\u107D\u107E\u107F\u1080\u1081"
        # Shan vowels and tones that are unique to Shan
        "\u1082\u1083\u1084\u1085\u1086\u1087\u1088\u1089\u108A"
    )
    
    # Additional characters allowed in Shan words
    shan_shared_chars = (
        "\u1004\u101E\u1010\u1011\u1015\u1019\u101A\u101B\u101C\u101D\u1022"  # Shared consonants
        "\u102D\u102E\u102F\u1030\u1031\u1035\u103A\u103B\u103C\u103D\u1038\u1062"  # Shared vowels and medials
    )
    
    # Numbers and punctuation
    shan_numbers = r"[႐-႙0-9]" if keep_numbers else r"[႐-႙]"  # Shan numbers ႐-႙
    shan_punctuations = "\u104a\u104b\ua9e6"  # ။၊ꧦ
    
    # Normalize Unicode
    text = unicodedata.normalize("NFC", text)
    
    # Split text into words
    words = text.split()
    cleaned_words = []
    
    for word in words:
        word = remove_emojis(word)
        
        # If the word contains Shan-specific chars or Shan numbers, process it
        if (any(char in word for char in shan_specific_chars) or
            any(char in word for char in shan_numbers) or
            re.search(shan_numbers, word)):
            
            # Create pattern of all allowed characters
            allowed_chars = f"{shan_specific_chars}{shan_shared_chars}{shan_numbers}{shan_punctuations}"
            pattern = rf"[^{allowed_chars}\s]"
            
            # Clean the word
            cleaned_word = re.sub(pattern, "", word)
            if cleaned_word:  # Only add non-empty words
                cleaned_words.append(cleaned_word)
    
    return " ".join(cleaned_words)

In [32]:
text = "ငဝ်ယိုဝ်ႈယိုတ်းယူႇ"
text2 = "ဢူးၺီႇၼတ်ႉဦးညီနပ်"

text_token = word_tokenize(text, engine="newmm")

for token in text_token:
    print(f"Token: {token}, Clean token: {clean_shan_text(token)}")

Token: ငဝ်, Clean token: 
Token: ယိုဝ်ႈယိုတ်း, Clean token: ယိုဝ်ႈယိုတ်း
Token: ယူႇ, Clean token: ယူႇ


In [19]:
@measure_time
def split_shan_chunks(text, min_len=20, max_len=50):
    # Initial text preprocessing
    # text = text.replace("၊", "၊ ").replace("။", "။ ").replace(" ၊", "၊ ").replace(" ။", "။ ").strip()
    print("Toknizing words...")
    tokens = word_tokenize(text, engine="newmm")
    
    chunks = []
    current_chunk = []
    current_length = 0
    
    print("Chunking text...")
    for token in tokens:
        print(token)
        token = clean_shan_text(token)
        token_len = len(token)
        
        # If adding token exceeds max_len, finalize current chunk
        if current_length + token_len > max_len and current_chunk:
            # If current chunk is too short, try to merge with previous
            if current_length < min_len and chunks:
                last_chunk = chunks.pop()
                chunks.append(last_chunk + ' ' + ''.join(current_chunk))
            else:
                chunks.append(''.join(current_chunk))
            current_chunk = [token]
            current_length = token_len
        else:
            # Add token to current chunk
            current_chunk.append(token)
            current_length += token_len
            
            # If we hit max_len exactly or have a good split point
            if (current_length >= min_len and token in (' ', '။')) or current_length == max_len:
                chunks.append(''.join(current_chunk))
                current_chunk = []
                current_length = 0
    
    # Handle remaining tokens
    if current_chunk:
        if current_length < min_len and chunks:
            last_chunk = chunks.pop()
            chunks.append(last_chunk + ' ' + ''.join(current_chunk))
        else:
            chunks.append(''.join(current_chunk))
    
    # Clean up whitespace in chunks
    chunks = [re.sub(r'\s+', ' ', chunk.strip()) for chunk in chunks if chunk.strip()]
    
    # Final validation pass
    print("Final validate chunking text...")
    final_chunks = []
    i = 0
    while i < len(chunks):
        current = chunks[i]
        
        # If chunk exceeds max_len, force split at last valid space
        if len(current) > max_len:
            split_idx = max_len
            while split_idx > 0 and current[split_idx] not in (' ', '။'):
                split_idx -= 1
            if split_idx > min_len:
                final_chunks.append(current[:split_idx].strip())
                chunks.insert(i + 1, current[split_idx:].strip())
            else:
                final_chunks.append(current)
        else:
            final_chunks.append(current)
        i += 1
    
    return final_chunks

# Test the function
text = "ဢူးၺီႇၼတ်ႉဦးညီနပ် ဢၼ်ယၢမ်ႈယၢဝ်းၵႂုႇလႆႈၶၢဝ်း 123 ၼၼ် တေၶဝ်ႈမႃးပဵၼ်သုၼ်ႇၼိုင်ႈၼႂ်းၸၢတ်ႈ ဢမ်ႇလႆႈႁၢမ်းၵၢတ်ႈၽူႈႁတ်းႁၢၼ်"
chunks = split_shan_chunks(text)
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1} (length {len(chunk)}): {chunk}")

Toknizing words...
Chunking text...
ဢူး
ၺီႇ
ၼတ်ႉ
ဦးညီနပ်
 
ဢၼ်
ယၢမ်ႈ
ယၢဝ်း
ၵ
ႂ
ု
ႇ
လႆႈ
ၶၢဝ်း
 
123
 
ၼၼ်
 
တေ
ၶဝ်ႈမႃး
ပဵၼ်
သုၼ်ႇ
ၼိုင်ႈ
ၼႂ်း
ၸၢတ်ႈ
 
ဢမ်ႇလႆႈ
ႁၢမ်း
ၵၢတ်ႈ
ၽူႈ
ႁတ်းႁၢၼ်
Final validate chunking text...
Function 'split_shan_chunks' took 0.0928 seconds to execute
Chunk 1 (length 45): ၺီႇၼတ်ႉဢၼ်ယၢမ်ႈၵႂႇလႆႈၶၢဝ်းၼၼ်ၶဝ်ႈမႃးပဵၼ်သုၼ်ႇ
Chunk 2 (length 43): ၼိုင်ႈၼႂ်းၸၢတ်ႈဢမ်ႇလႆႈႁၢမ်းၵၢတ်ႈၽူႈႁတ်းႁၢၼ်


In [ ]:
@measure_time
def generate_images_from_huggingface(dataset_repo, chunk_size, fonts, output_dir):
    generator = OCRDataGenerator(font_paths=fonts)
    chunk_count = 0
    
    # Load dataset from Hugging Face
    dataset = load_dataset(dataset_repo, split="train")
    contents = dataset["content"]

    print("Generate images...")
    for content in contents:
        content = content.strip()
        content = clean_shan_text(content, keep_numbers=True)

        texts = split_shan_chunks(content)

        for text in texts:
            if len(text) < 1:
                continue
            
            text = re.sub(r"(^။)|(^၊)", "", text) # remove start ၊, ။
            text = text.strip()

            image, metadata = generator.generate_image(
                text=text,
                min_font_size=24,
                max_font_size=48,
                horizontal_padding=40,
                vertical_padding=20,
                min_height=64,
                add_noise=True,
                random_transform=False
            )
            print(f"Text: {text}")
            print(f"Image size: {metadata['image_size']}\n")

            ts = time.time()

            # Save TIF
            image.save(f"{output_dir}/{ts}.tif")

            # Save TXT
            with open(f"{output_dir}/{ts}.gt.txt", "w", encoding='utf-8') as text_file:
                text_file.write(text)

            print(f"Saved image for word: {text}")

            chunk_count += 1
            print(f"chunk size: {chunk_count}")

            if chunk_count > chunk_size:
                return
        
        print(f"Total chunk size: {chunk_count}")


In [ ]:
output_dir = "../data/shn-ground-truth"
fonts = [
    "Shan.ttf",
    "PangLong.ttf",
    "GreatHorKham_Taunggyi.ttf",
    "mmrtext.ttf",
    "Pyidaungsu.ttf"
]

huggingface_datasets_repo = [
    "NorHsangPha/shan-novel-tainovel_com",
    "NorHsangPha/shan-news-shannews_org",
    # "NorHsangPha/shan-news-taifreedom_com",
    # "NorHsangPha/shan-news-shanhumanrights_org",
    # "NorHsangPha/shan-news-ssppssa_org",
]

for repo in huggingface_datasets_repo:
    generate_images_from_huggingface(dataset_repo=repo, chunk_size=200000, fonts=fonts, output_dir=output_dir)